# Interacciones e impactos sociales

A diferencia del trabajo realizado en [1] en el que cada celda representaba una región, consideraremos a cada división como un único individuo que será dotado de un conjunto de cualidades como estado de salud, edad, vecinos, etc. Estas cualidades vendrán dadas por las necesidades del modelo que estemos desarrollando, por ejemplo en los modelos clásicos no será necesario dotar de edades a las células, pero si tendremos que tener en cuenta sus vecindades y sus estados de salud.

Pensemos por un momento en las características que hay detrás de relación cercana entre individuos o células. Denotaremos la relación entre células con el símbolo $\thicksim$ y una vez dicho esto tenemos que:

*   Todas las células están en contacto con ellas mismas, por lo que para cada célula $x$ se cumple $x \thicksim x$.
*   Si una célula estuviera en contacto con alguna otra entonces esa célula estaría en contacto con la primera, es decir, $x\thicksim y$ implica $y\thicksim x$.
*   Si una célula interactúa con otras dos no implica necesariamente que estas interactúen entre sí, por lo que $x\thicksim y$ y $x\thicksim z$ no implican que $y\thicksim z$.

**Ejemplo:** Consideremos por ejemplo el conjunto $A=\{a,b,c,d\}$ y a la sub-colección $\mathcal{B} = \{\{a\},\{b\},\{c\},\{d\},\{a,c,d\}, \{b,c\}, \{b,d\}\}$ de $\mathcal{P}(A)$. Claramente se obtienen las siguientes relaciones

$$\begin{array}{cccc}
    a\thicksim a, & b\thicksim b, & c\thicksim c, & d\thicksim d, \\ 
    a\thicksim c, & a\thicksim d, & b\thicksim c, & b\thicksim d,
\end{array}$$

junto con sus relaciones simétricas equivalentes. Esta noción de relación de interacción entre células puede ir un poco más allá. Pensemos por un momento en el impacto que puede tener un comportamiento sobre la célula $b$ en la célula $a$, si bien estas dos celdas no interactúan entre sí, la relación que cada una de ellas tiene con las células $c$ y $d$ puede tener un impacto sobre la otra. Con lo cual definimos la siguiente relación de interacción:

**Definición:** Definimos el *grado de impacto* entre dos puntos $a$ y $b$ como la menor cantidad de interacciones necesaria para llegar de $a$ a $b$. Para reconocer el grado de impacto entre dos puntos usaremos la notación $a\thicksim_n b$ donde $n\in\mathbb{N}$ denota la menor cantidad de interacciones entre $a$ y $b$.

Si retomamos el ejemplo 3.1.1 podremos identificar los siguientes grados de impacto:

$$\begin{array}{cccc}
    a\thicksim_0 a, & b\thicksim_0 b, & c\thicksim_0 c, & d\thicksim_0 d, \\ 
    a\thicksim_1 c, & a\thicksim_1 d, & b\thicksim_1 c, & b\thicksim_1 d, \\
    a\thicksim_2 b, & c\thicksim_2 d.
\end{array}$$

De la definición anterior se deduce el siguiente resultado:

**Teorema:** Los grados de impacto de una célula $x$ definen un sistema fundamental de vecindades en la topología discreta.

Demostración: Sea $x\in\mathcal{L}$ y $\mathcal{V}(x)$ una familia de vecindades de $x$. Defina el conjunto $A_0=\{x\}$ como el conjunto de puntos con grado de impacto con $x$ es igual a cero y a continuación defina de manera recursiva a los conjuntos $A_k$ cuyos elementos tienen grado de impacto con $x$ sea igual o menor a $k$. Claramente $A_i\subseteq A_j$ para $0\leq i\leq j$ y de ese modo $A_i\in\mathcal{V}(x)$ para $i=0,1,\cdots,n$.

Defina la colección $\mathcal{A}=\{A_0,A_1,\cdots,A_k,\cdots,A_N\}$ como la familia de conjuntos encajados definidos por el grado de impacto con $x$. Dado que por hipótesis estamos sobre la topología discreta podemos afirmar que para todo $V\in\mathcal{V}(x)$ el conjunto $A_0\subseteq V$ y de ese modo por la definición 2.4.8 concluimos que la colección $\mathcal{A}$ es un sistema fundamental de vecindades de $x$.

A continuación mostraremos algunas de las propiedades del conjunto $\mathcal{A}$ definido en la demostración anterior:

**Proposición:** Sea $x\in\mathcal{L}$ una célula y sea $\mathcal{A}$ la familia de conjuntos encajados definidos por el grado de impacto con $x$. Se cumplen las siguientes propiedades:

1.  El conjunto $\mathcal{A}$ posee elemento mínima igual a $A_0=\{x\}$,
2.  $\mathcal{A}$ es un conjunto ordenado finito con el orden de la contenencia,
3.  $\mathcal{L}$ es un espacio $T_0$, y
4.  $\mathcal{L}$ es un espacio uno-numerable.

A continuación se presentan ejemplos para el manejo e implementación de los sistemas fundamentales de vecindades que describen las interacciones sobre algún sistema:

In [1]:
from EpidemiologicalModels.epidemiologicalModelsInCA import *

# Ejemplo 1: Sistemas de vecindades básicos generados a partir de vecindades especificas o aleatoreamente.

# Ejemplo 1.1: 
cellSpace_ex1 = CellSpace(3,3)  # Espacio de células con dimensión igual a 3x3
GenerateNeighborhoodSystem(cellSpace_ex1)  # Sistema de vecindades generado aleatoriamente

[[[0, 0],
  array([[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]])],
 [[0, 1],
  array([[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]])],
 [[0, 2],
  array([[1., 1., 0.],
         [1., 1., 1.],
         [1., 1., 1.]])],
 [[1, 0],
  array([[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]])],
 [[1, 1],
  array([[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]])],
 [[1, 2],
  array([[1., 1., 1.],
         [1., 1., 0.],
         [1., 1., 1.]])],
 [[2, 0],
  array([[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]])],
 [[2, 1],
  array([[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]])],
 [[2, 2],
  array([[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]])]]

In [2]:
# Ejemplo 1.2:
GenerateNeighborhoodSystem(cellSpace_ex1,"Moore")  # Sistema de vecindades generado a partir de las vecindades de Moore

[[[0, 0],
  array([[0., 0., 1.],
         [0., 0., 1.],
         [1., 1., 1.]])],
 [[0, 1],
  array([[0., 0., 0.],
         [0., 0., 0.],
         [1., 1., 1.]])],
 [[0, 2],
  array([[1., 0., 0.],
         [1., 0., 0.],
         [1., 1., 1.]])],
 [[1, 0],
  array([[0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.]])],
 [[1, 1],
  array([[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]])],
 [[1, 2],
  array([[1., 0., 0.],
         [1., 0., 0.],
         [1., 0., 0.]])],
 [[2, 0],
  array([[1., 1., 1.],
         [0., 0., 1.],
         [0., 0., 1.]])],
 [[2, 1],
  array([[1., 1., 1.],
         [0., 0., 0.],
         [0., 0., 0.]])],
 [[2, 2],
  array([[1., 1., 1.],
         [1., 0., 0.],
         [1., 0., 0.]])]]

In [3]:
# Ejemplo 1.3:
GenerateNeighborhoodSystem(cellSpace_ex1,"Von Neumann")  # Sistema de vecindades generado a partir de las vecindades de Von Neumann

[[[0, 0],
  array([[0., 0., 1.],
         [0., 1., 1.],
         [1., 1., 1.]])],
 [[0, 1],
  array([[0., 0., 0.],
         [1., 0., 1.],
         [1., 1., 1.]])],
 [[0, 2],
  array([[1., 0., 0.],
         [1., 1., 0.],
         [1., 1., 1.]])],
 [[1, 0],
  array([[0., 1., 1.],
         [0., 0., 1.],
         [0., 1., 1.]])],
 [[1, 1],
  array([[1., 0., 1.],
         [0., 0., 0.],
         [1., 0., 1.]])],
 [[1, 2],
  array([[1., 1., 0.],
         [1., 0., 0.],
         [1., 1., 0.]])],
 [[2, 0],
  array([[1., 1., 1.],
         [0., 1., 1.],
         [0., 0., 1.]])],
 [[2, 1],
  array([[1., 1., 1.],
         [1., 0., 1.],
         [0., 0., 0.]])],
 [[2, 2],
  array([[1., 1., 1.],
         [1., 1., 0.],
         [1., 0., 0.]])]]

In [4]:
# Ejemplo 2: Matrices de interacción definidos manualmente

import numpy as np

x00 = [[0,0], np.array([[0,1,1], [1,2,1], [1,2,1]])]
x01 = [[0,1], np.array([[1,0,0], [1,2,1], [1,2,1]])]
x02 = [[0,2], np.array([[1,0,0], [1,2,1], [1,2,1]])]
x10 = [[1,0], np.array([[1,1,1], [0,2,1], [0,2,1]])]
x11 = [[1,1], np.array([[2,2,2], [2,0,2], [2,0,2]])]
x12 = [[1,2], np.array([[1,1,1], [1,2,0], [1,2,1]])]
x20 = [[2,0], np.array([[1,1,1], [0,2,1], [0,2,1]])]
x21 = [[2,1], np.array([[2,2,2], [2,0,2], [2,0,2]])]
x22 = [[2,2], np.array([[1,1,1], [1,2,1], [1,2,0]])]

neighborhoodSystem_ex2 = [x00, x01, x02, x10, x11, x12, x20, x21, x22]
neighborhoodSystem_ex2

[[[0, 0],
  array([[0, 1, 1],
         [1, 2, 1],
         [1, 2, 1]])],
 [[0, 1],
  array([[1, 0, 0],
         [1, 2, 1],
         [1, 2, 1]])],
 [[0, 2],
  array([[1, 0, 0],
         [1, 2, 1],
         [1, 2, 1]])],
 [[1, 0],
  array([[1, 1, 1],
         [0, 2, 1],
         [0, 2, 1]])],
 [[1, 1],
  array([[2, 2, 2],
         [2, 0, 2],
         [2, 0, 2]])],
 [[1, 2],
  array([[1, 1, 1],
         [1, 2, 0],
         [1, 2, 1]])],
 [[2, 0],
  array([[1, 1, 1],
         [0, 2, 1],
         [0, 2, 1]])],
 [[2, 1],
  array([[2, 2, 2],
         [2, 0, 2],
         [2, 0, 2]])],
 [[2, 2],
  array([[1, 1, 1],
         [1, 2, 1],
         [1, 2, 0]])]]